In [1]:
from mmdeploy.apis import torch2onnx
# from mmdeploy.backend.sdk.export_info import export2SDK
import torch
import onnx, onnxruntime
from mmdeploy.apis.utils import build_task_processor
from mmdeploy.utils import get_input_shape, load_config
import torch
import onnx
import torch
from onnx2torch import convert

/home/meysam/anaconda3/lib/python3.12/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [2]:
img = 'demo/531__0_1200.png'
work_dir = 'mmdeploy_models/mmdet/onnx'
save_file = 'tinydet_l_aitod.onnx'
deploy_cfg = '../mmdeploy/configs/mmdet/detection/detection_onnxruntime_custom.py'
model_cfg = 'configs/tinydet/tinydet_L_aitod.py'
model_checkpoint = 'work_dirs/tinydet_L_aitod/epoch_566.pth'
device = 'cuda'

In [3]:
deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)
model_cfg['model']['test_cfg']['rcnn'] = None
model_cfg['model']['test_cfg']['rpn']['nms_pre'] = -1
deploy_cfg['input_shape'] = (512,512)

In [4]:
# deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)
task_processor = build_task_processor(model_cfg, deploy_cfg, device)
torch_model = task_processor.build_pytorch_model(model_checkpoint)

01/27 11:30:49 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
01/27 11:30:49 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "mmdet_tasks" registry tree. As a workaround, the current "mmdet_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/meysam/anaconda3/lib/python3.12/site-packages/mmengine/runner/checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_lo

--------------------------
{'type': 'PSROIAlign_ori', 'roi_size': 7, 'sampling_ratio': 2, 'pooled_dim': 5}
PSROIAlign_ori
--------------------------
{'type': 'PSROIAlign_ori', 'roi_size': 7, 'sampling_ratio': 2, 'pooled_dim': 5}
PSROIAlign_ori
Loads checkpoint by local backend from path: work_dirs/tinydet_L_aitod/epoch_566.pth


/home/meysam/test-apps/mmdetection/mmdet/models/task_modules/builder.py:24: UserWarning: ``build_iou_calculator`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn(
/home/meysam/test-apps/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


In [5]:
t_inp = torch.randn(1,3,512,512).cuda()
input_shape = get_input_shape(deploy_cfg)
model_inputs, _ = task_processor.create_input(img, input_shape)
batch_data_samples = model_inputs.get('data_samples')


In [10]:
x = torch.randn(1,3,512,512).cuda()
feat = torch_model.extract_feat(x) # backbone and necks


In [11]:
for ff in feat:
    print(f"{ff.shape}")

torch.Size([1, 245, 128, 128])
torch.Size([1, 245, 64, 64])
torch.Size([1, 245, 32, 32])
torch.Size([1, 245, 16, 16])
torch.Size([1, 245, 8, 8])


In [63]:
rpn_results_list = torch_model.rpn_head.predict(
        feat, batch_data_samples, rescale=False)
roi_outs = torch_model.roi_head.forward(feat, rpn_results_list,
                                         batch_data_samples)

In [73]:
rpn_results_list[0].bboxes.shape

torch.Size([3000, 4])

In [49]:
outs = torch_model.rpn_head(feat)
# BaseDenseHead . predict_by_feat(*outs


In [58]:
cls_scores = outs[0]
bbox_preds = outs[1]

In [59]:
for cc in cls_scores:
    print(cc.shape)

torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 8, 8])


In [60]:
for bb in bbox_preds:
    print(bb.shape)

torch.Size([1, 12, 128, 128])
torch.Size([1, 12, 64, 64])
torch.Size([1, 12, 32, 32])
torch.Size([1, 12, 16, 16])
torch.Size([1, 12, 8, 8])


In [67]:
roi_outs[0].shape

torch.Size([3000, 10])

In [68]:
roi_outs[1].shape

torch.Size([3000, 4])

In [32]:
roi_outs = torch_model.roi_head.forward(x, rpn_results_list, batch_data_samples)

AttributeError: 'list' object has no attribute 'bboxes'

In [34]:
torch_model.roi_head.bbox_roi_extractor

SingleRoIExtractorModified(
  (roi_layers): ModuleList(
    (0): PSRoIAlign(output_size=7, spatial_scale=0.25, sampling_ratio=2)
    (1): PSRoIAlign(output_size=7, spatial_scale=0.125, sampling_ratio=2)
    (2): PSRoIAlign(output_size=7, spatial_scale=0.0625, sampling_ratio=2)
  )
)

In [21]:
len(rpn_results_list)

2

In [24]:
len(rpn_results_list[0])

5

In [25]:
len(rpn_results_list[1])

5

In [26]:
rpn_results_list[0][0].shape

torch.Size([1, 3, 128, 128])

In [30]:
rpn_results_list[0][1].shape

torch.Size([1, 3, 64, 64])

In [29]:
rpn_results_list[0][2].shape

torch.Size([1, 3, 32, 32])

In [28]:
rpn_results_list[0][3].shape

torch.Size([1, 3, 16, 16])

In [27]:
rpn_results_list[0][4].shape

torch.Size([1, 3, 8, 8])

In [ ]:
        rpn_results_list = self.torch_model.rpn_head.predict(
                x, batch_data_samples, rescale=False)
        labels = rpn_results_list[0].get('labels')
        bboxes = rpn_results_list[0].get('bboxes')
        scores = rpn_results_list[0].get('scores')

In [6]:
oo = torch_model._forward(x, batch_data_samples)

NameError: name 'x' is not defined

In [7]:
torch_model.rpn_head

TinyRPN(
  (loss_cls): CrossEntropyLoss(avg_non_ignore=False)
  (loss_bbox): SmoothL1Loss()
  (rpn_conv): Sequential(
    (0): Conv2d(245, 245, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=245)
    (1): Conv2d(245, 245, kernel_size=(1, 1), stride=(1, 1), groups=49)
  )
  (rpn_cls): Conv2d(245, 3, kernel_size=(1, 1), stride=(1, 1))
  (rpn_reg): Conv2d(245, 12, kernel_size=(1, 1), stride=(1, 1))
)
init_cfg={'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}

In [8]:
torch_model.roi_head

StandardRoIHead(
  (bbox_roi_extractor): SingleRoIExtractorModified(
    (roi_layers): ModuleList(
      (0): PSRoIAlign(output_size=7, spatial_scale=0.25, sampling_ratio=2)
      (1): PSRoIAlign(output_size=7, spatial_scale=0.125, sampling_ratio=2)
      (2): PSRoIAlign(output_size=7, spatial_scale=0.0625, sampling_ratio=2)
    )
  )
  (bbox_head): SharedFCBBoxHeadModified(
    (loss_cls): CrossEntropyLoss(avg_non_ignore=False)
    (loss_bbox): SmoothL1Loss()
    (fc_cls): Linear(in_features=1024, out_features=10, bias=True)
    (fc_reg): Linear(in_features=1024, out_features=4, bias=True)
    (shared_convs): ModuleList()
    (shared_fcs): ModuleList(
      (0): Linear(in_features=245, out_features=1024, bias=True)
    )
    (cls_convs): ModuleList()
    (cls_fcs): ModuleList()
    (reg_convs): ModuleList()
    (reg_fcs): ModuleList()
    (relu): ReLU(inplace=True)
  )
  init_cfg=[{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'overri

In [9]:
oo[2].shape

NameError: name 'oo' is not defined

In [10]:
out[0].get('pred_instances').get('bboxes')

tensor([[1.3364e+02, 1.6008e-01, 1.5197e+02, 1.1681e+01],
        [0.0000e+00, 7.3067e+02, 2.6543e+02, 8.0000e+02],
        [2.8536e+00, 7.1339e+02, 1.7606e+02, 8.0000e+02],
        ...,
        [1.5132e+02, 2.4669e+01, 1.6611e+02, 4.0586e+01],
        [4.6649e+02, 2.9237e+01, 4.7864e+02, 4.8260e+01],
        [3.5899e+02, 1.2757e+01, 3.8208e+02, 2.8822e+01]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [48]:
class ModelForExport(torch.nn.Module):
    def __init__(self):
        super(ModelForExport, self).__init__()
        self.torch_model = task_processor.build_pytorch_model(model_checkpoint)
        # torch_model.eval()
    def forward(self, x):
        results = ()
        x = self.torch_model.extract_feat(x)
        rpn_results_list = self.torch_model.rpn_head.predict(
                x, batch_data_samples, rescale=False)
        # labels = rpn_results_list[0].get('labels')
        # bboxes = rpn_results_list[0].get('bboxes')
        # scores = rpn_results_list[0].get('scores')

        # print(len(rpn_results_list))
        # print(rpn_results_list[0].keys())
        # ['labels', 'bboxes', 'scores']
        # labels = 
        
        roi_outs = self.torch_model.roi_head.forward(x, rpn_results_list,
                                         batch_data_samples)
        return roi_outs[0], roi_outs[1]
        # return labels, bboxes, scores
    # def forward(self, x):
        # out = self.torch_model.forward(x, batch_data_samples)
        # return out[0][0], out[0][1]
        # out = torch_model.predict2(t_inp, batch_data_samples)
        # return out[0].get('pred_instances').get('bboxes'), out[0].get('pred_instances').get('scores')
        

In [49]:
model_to_export = ModelForExport()
model_to_export.eval()
model_to_export.cuda()
x = torch.randn(1,3,512,512).cuda()

The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv2.0.weight, conv2.1.weight, conv2.1.bias, conv2.1.running_mean, conv2.1.running_var, conv2.1.num_batches_tracked, linear3.0.weight, linear3.0.bias, linear4.weight, linear4.bias

missing keys in source state_dict: blocks.18.se.se.1.weight, blocks.18.se.se.3.weight, blocks.18.conv1.weight, blocks.18.bn1.weight, blocks.18.bn1.bias, blocks.18.bn1.running_mean, blocks.18.bn1.running_var, blocks.18.conv2.weight, blocks.18.bn2.weight, blocks.18.bn2.bias, blocks.18.bn2.running_mean, blocks.18.bn2.running_var, blocks.18.conv3.weight, blocks.18.bn3.weight, blocks.18.bn3.bias, blocks.18.bn3.running_mean, blocks.18.bn3.running_var, blocks.19.se.se.1.weight, blocks.19.se.se.3.weight, blocks.19.conv1.weight, blocks.19.bn1.weight, blocks.19.bn1.bias, blocks.19.bn1.running_mean, blocks.19.bn1.running_var, blocks.19.conv2.weight, blocks.19.bn2.weight, blocks.19.bn2.bias, blocks.19.bn2.running_mean, blocks.19.

--------------------------
{'type': 'PSROIAlign_ori', 'roi_size': 7, 'sampling_ratio': 2, 'pooled_dim': 5}
PSROIAlign_ori
--------------------------
{'type': 'PSROIAlign_ori', 'roi_size': 7, 'sampling_ratio': 2, 'pooled_dim': 5}
PSROIAlign_ori
Loads checkpoint by local backend from path: work_dirs/tinydet_L_aitod/epoch_566.pth


In [50]:
# labels, bboxes, scores = model_to_export(x)

In [51]:
roi_outs = model_to_export(x)

In [52]:
roi_outs[0].shape

torch.Size([3000, 10])

In [53]:
roi_outs[1].shape

torch.Size([3000, 4])

In [54]:
torch_model.roi_head

StandardRoIHead(
  (bbox_roi_extractor): SingleRoIExtractorModified(
    (roi_layers): ModuleList(
      (0): PSRoIAlign(output_size=7, spatial_scale=0.25, sampling_ratio=2)
      (1): PSRoIAlign(output_size=7, spatial_scale=0.125, sampling_ratio=2)
      (2): PSRoIAlign(output_size=7, spatial_scale=0.0625, sampling_ratio=2)
    )
  )
  (bbox_head): SharedFCBBoxHeadModified(
    (loss_cls): CrossEntropyLoss(avg_non_ignore=False)
    (loss_bbox): SmoothL1Loss()
    (fc_cls): Linear(in_features=1024, out_features=10, bias=True)
    (fc_reg): Linear(in_features=1024, out_features=4, bias=True)
    (shared_convs): ModuleList()
    (shared_fcs): ModuleList(
      (0): Linear(in_features=245, out_features=1024, bias=True)
    )
    (cls_convs): ModuleList()
    (cls_fcs): ModuleList()
    (reg_convs): ModuleList()
    (reg_fcs): ModuleList()
    (relu): ReLU(inplace=True)
  )
  init_cfg=[{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'overri

In [55]:
# if not os.path.isfile(ONNX_FILE_PATH):
ONNX_FILE_PATH = work_dir + '/'+ save_file
torch.onnx.export(model_to_export, (x), ONNX_FILE_PATH, input_names=["x"], 
                output_names=["bboxes","classes"], export_params=True) #external_data=True , opset_version = 7

/home/meysam/test-apps/mmdetection/mmdet/models/dense_heads/base_dense_head.py:393: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert cls_score.size()[-2:] == bbox_pred.size()[-2:]
/home/meysam/test-apps/mmdetection/mmdet/models/utils/misc.py:336: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  num_topk = min(topk, valid_idxs.size(0))
/home/meysam/test-apps/mmdetection/mmdet/models/task_modules/coders/delta_xywh_bbox_coder.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flo

UnsupportedOperatorError: ONNX export failed on an operator with unrecognized namespace torchvision::ps_roi_align. If you are trying to export a custom operator, make sure you registered it with the right domain and version.

In [21]:
# if not os.path.isfile(ONNX_FILE_PATH):
ONNX_FILE_PATH = work_dir + '/'+ save_file
torch.onnx.export(model_to_export, (x), ONNX_FILE_PATH, input_names=["x"], 
                output_names=["bboxes","scores","scores"], export_params=True) #external_data=True , opset_version = 7

/home/meysam/test-apps/mmdetection/mmdet/models/dense_heads/base_dense_head.py:393: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert cls_score.size()[-2:] == bbox_pred.size()[-2:]
/home/meysam/test-apps/mmdetection/mmdet/models/utils/misc.py:336: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  num_topk = min(topk, valid_idxs.size(0))
/home/meysam/test-apps/mmdetection/mmdet/models/task_modules/coders/delta_xywh_bbox_coder.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flo

1


In [9]:
help(torch.onnx.export)

Help on function export in module torch.onnx:

export(model: 'torch.nn.Module | torch.export.ExportedProgram | torch.jit.ScriptModule | torch.jit.ScriptFunction', args: 'tuple[Any, ...]' = (), f: 'str | os.PathLike | None' = None, *, kwargs: 'dict[str, Any] | None' = None, export_params: 'bool' = True, verbose: 'bool | None' = None, input_names: 'Sequence[str] | None' = None, output_names: 'Sequence[str] | None' = None, opset_version: 'int | None' = None, dynamic_axes: 'Mapping[str, Mapping[int, str]] | Mapping[str, Sequence[int]] | None' = None, keep_initializers_as_inputs: 'bool' = False, dynamo: 'bool' = False, external_data: 'bool' = True, dynamic_shapes: 'dict[str, Any] | tuple[Any, ...] | list[Any] | None' = None, report: 'bool' = False, verify: 'bool' = False, profile: 'bool' = False, dump_exported_program: 'bool' = False, artifacts_dir: 'str | os.PathLike' = '.', fallback: 'bool' = False, training: '_C_onnx.TrainingMode' = <TrainingMode.EVAL: 0>, operator_export_type: '_C_onnx.

torch.Size([3000, 9])

torch.Size([3000, 32])

In [11]:
# if not os.path.isfile(ONNX_FILE_PATH):
ONNX_FILE_PATH = work_dir + '/'+ save_file
torch.onnx.export(model_to_export, (x), ONNX_FILE_PATH, input_names=["x"], 
                output_names=["bboxes","scores"], export_params=True)

0
1
2
3
4
(400, 400)
nms


In [12]:
# 1. convert model to onnx
# t_inp = torch.randn(1,3,800,800)
# t_inp = torch.Tensor(t_inp)
torch2onnx(img, work_dir, save_file, deploy_cfg, model_cfg,
           model_checkpoint, device)

Loads checkpoint by local backend from path: work_dirs/faster-rcnn_r50_rfla_kld_1x_aitod/epoch_12.pth
12/25 08:28:05 - mmengine - WARNING - DeprecationWarning: get_onnx_config will be deprecated in the future. 
12/25 08:28:05 - mmengine - INFO - Export PyTorch model to ONNX: mmdeploy_models/mmdet/onnx/rfla.onnx.
12/25 08:28:05 - mmengine - WARNING - Can not find torch.nn.functional._scaled_dot_product_attention, function rewrite will not be applied
12/25 08:28:05 - mmengine - WARNING - Can not find mmdet.models.utils.transformer.PatchMerging.forward, function rewrite will not be applied


/home/meysam/anaconda3/lib/python3.12/site-packages/mmdeploy/codebase/mmdet/models/detectors/two_stage.py:71: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  img_shape = [int(val) for val in img_shape]
/home/meysam/anaconda3/lib/python3.12/site-packages/mmdeploy/codebase/mmdet/models/detectors/two_stage.py:71: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  img_shape = [int(val) for val in img_shape]
/home/meysam/anaconda3/lib/python3.12/site-packages/mmdeploy/core/optimizers/function_marker.py:160: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We c

RuntimeError: Trying to create tensor with negative dimension -1: [1, -1, 1]

In [ ]:
model_name = work_dir + '/' + save_file
onnx_model = onnx.load(model_name)

In [ ]:
# onnx_model.graph.output

In [ ]:
backend_model = [work_dir + '/' + save_file]

In [ ]:
# read deploy_cfg and model_cfg
deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)

In [ ]:
# build task and backend model
task_processor = build_task_processor(model_cfg, deploy_cfg, device)
model = task_processor.build_backend_model(backend_model)


In [ ]:

task_processor = build_task_processor(model_cfg, deploy_cfg, device)

# process input image
input_shape = get_input_shape(deploy_cfg)
model_inputs, _ = task_processor.create_input(img, input_shape)


In [ ]:
deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)
task_processor = build_task_processor(model_cfg, deploy_cfg, device)
torch_model = task_processor.build_pytorch_model(model_checkpoint)

In [ ]:
type(torch_model)

In [ ]:
t_inp

In [ ]:
out[0].keys()

In [ ]:
out[0].get('pred_instances')

In [ ]:
input_shape = get_input_shape(deploy_cfg)

In [ ]:
input_shape = get_input_shape(deploy_cfg)
model_inputs, _ = task_processor.create_input(img, input_shape)
model_inputs.get('data_samples')

In [ ]:
model_inputs.get('data_samples')

In [ ]:
with torch.no_grad():
    result = model.test_step(model_inputs)

In [ ]:
result[0].get('pred_instances').scores.shape

In [ ]:
x = torch.randn(1,3,800,800)
feat = torch_model.extract_feat(x)

In [ ]:
len(feat)

In [ ]:
for f in feat:
    print(f.shape)

In [ ]:
feat[0].shape

In [13]:
model_name = work_dir + '/' + save_file
onnx_model = onnx.load(model_name)

In [14]:
type(onnx_model)

onnx.onnx_ml_pb2.ModelProto

In [19]:
for i, g in enumerate(onnx_model.graph.node):
    # print(g)
    if "NonZero" in  g.op_type:
        print(i , g)

677 input: "/Equal_output_0"
output: "/NonZero_output_0"
name: "/NonZero"
op_type: "NonZero"

690 input: "/Expand_16_output_0"
output: "/NonZero_1_output_0"
name: "/NonZero_1"
op_type: "NonZero"

724 input: "/Equal_2_output_0"
output: "/NonZero_2_output_0"
name: "/NonZero_2"
op_type: "NonZero"

737 input: "/Expand_18_output_0"
output: "/NonZero_3_output_0"
name: "/NonZero_3"
op_type: "NonZero"

778 input: "/Equal_4_output_0"
output: "/NonZero_4_output_0"
name: "/NonZero_4"
op_type: "NonZero"

791 input: "/Expand_20_output_0"
output: "/NonZero_5_output_0"
name: "/NonZero_5"
op_type: "NonZero"

832 input: "/Equal_6_output_0"
output: "/NonZero_6_output_0"
name: "/NonZero_6"
op_type: "NonZero"

845 input: "/Expand_22_output_0"
output: "/NonZero_7_output_0"
name: "/NonZero_7"
op_type: "NonZero"

886 input: "/Equal_8_output_0"
output: "/NonZero_8_output_0"
name: "/NonZero_8"
op_type: "NonZero"

899 input: "/Expand_24_output_0"
output: "/NonZero_9_output_0"
name: "/NonZero_9"
op_type: "NonZer

In [16]:
onnx_model.graph.node[460]

input: "/Transpose_2_output_0"
input: "/Constant_96_output_0"
output: "/Reshape_15_output_0"
name: "/Reshape_15"
op_type: "Reshape"
attribute {
  name: "allowzero"
  i: 0
  type: INT
}

In [17]:
torch_model

FasterRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tru